## Part 1: Linear Regression

In [ ]:
## Importing Data
# tf.__version__

NameError: ignored

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

def loadData():
    with np.load('notMNIST.npz') as data :
        Data, Target = data ['images'], data['labels']
        posClass = 2
        negClass = 9
        dataIndx = (Target==posClass) + (Target==negClass)
        Data = Data[dataIndx]/255.
        Target = Target[dataIndx].reshape(-1, 1)
        Target[Target==posClass] = 1
        Target[Target==negClass] = 0
        np.random.seed(421)
        randIndx = np.arange(len(Data))
        np.random.shuffle(randIndx)
        Data, Target = Data[randIndx], Target[randIndx]
        trainData, trainTarget = Data[:3500], Target[:3500]
        validData, validTarget = Data[3500:3600], Target[3500:3600]
        testData, testTarget = Data[3600:], Target[3600:]
    return trainData, validData, testData, trainTarget, validTarget, testTarget

In [ ]:
trainData, validData, testData, trainTarget, validTarget, testTarget = loadData()
print(trainTarget)

[[0]
 [1]
 [1]
 ...
 [0]
 [1]
 [0]]


In [ ]:
# Input Variables
#     W: <features_x * features_y, 1>
#     b: scaler
#     x: <n_samples, features_x * features_y>
#     y: <n_samples, 1>
#     reg: scaler

def reshape_data(data):
    '''
    Reshapes data x to <n_samples, n_features>
    
    Input:
        Data <n_samples, features_x, features_y>
    
    Output: 
        Data_aug <n_samples, features_x * features_y>,
        The data matrix, flattened.
    '''
    
    data_aug = np.reshape(data, (data.shape[0], data.shape[1]*data.shape[2]))
    return data_aug

trainData, validData, testData= map(reshape_data, (trainData, validData, testData))

print(trainData.shape)

(3500, 784)


In [ ]:
W = 2 * np.random.random_sample((784, 1))-1
b = 0
x = trainData
y = trainTarget
reg = 0.01

# USE THIS ONE
def MSE(W, b, x, y, reg):
    e_in = np.matmul(x, W) + b - y
    L_D = np.sum(np.square(np.linalg.norm(e_in))) / x.shape[0]
    L_W = np.square(np.linalg.norm(W)) * reg / 2
    return L_D + L_W

def MSECheck(W, b, x, y, reg):
    onesCol = np.ones([x.shape[0], 1])
    xTilde = np.concatenate((onesCol, x), axis=1)
    wTilde = np.insert(W, 0, b).reshape(-1, 1)
    e_in = np.matmul(xTilde, wTilde) - y
    L_D = np.square(np.linalg.norm(e_in)) / x.shape[0]
    L_W = reg / 2 * np.square(np.linalg.norm(W))
    return L_D + L_W

def MSECheck2(W, b, x, y, reg):
    error = np.matmul(x,W) + b - y
    mse = (np.sum(error*error))/((np.shape(y)[0])) + reg/2*np.sum(W*W)
    return mse

print(MSECheck(W, b, x, y, reg))
print(MSECheck2(W, b, x, y, reg))
print(MSE(W, b, x, y, reg))

211.30876015088901
211.308760150889
211.30876015088901


In [ ]:
W = 2 * np.random.random_sample((784, 1))-1
b = 0
x = trainData
y = trainTarget
reg = 0.01

# USE THIS ONE
def gradMSE(W, b, x, y, reg):
    e_in = np.matmul(x, W) + b - y
    dL_D_dw = 2 * (np.matmul(np.transpose(x), e_in)) / x.shape[0]
    dL_W_dw = 2 * reg * W
    dL_D_db = 2 * np.sum(e_in) / x.shape[0]
    return dL_D_dw + dL_W_dw, dL_D_db

def gradMSECheck(W, b, x, y, reg):
    error = np.matmul(x,W) + b - y
    grad_mse_W = 2 * (np.matmul(np.transpose(x),error)/(np.shape(y)[0])) + 2*reg*W
    grad_mse_b = 2 * ((np.sum(error))/(np.shape(y)[0]))
    return grad_mse_W, grad_mse_b

print(gradMSE(W, b, x, y.copy(), reg)[0][0:5])
print(gradMSECheck(W, b, x, y, reg)[0][0:5])

[[0.33643829]
 [0.44432914]
 [0.65383208]
 [0.89754309]
 [1.2091064 ]]
[[0.33643829]
 [0.44432914]
 [0.65383208]
 [0.89754309]
 [1.2091064 ]]


In [ ]:
# def gradMSE(W, b, x, y, reg):
    # '''
    # Inputs:
    #   W <28, 28>
    #   b scaler
    #   x <3500, 28, 28>
    #   y <3500, 1>
    #   reg scaler
    # My variables:
    #   X_aug <N, d+1>
    #   w_aug <d+1, 1>
    #   y <N, 1>
    #   reg scaler
    # Output:
    #   grad <d+1, 1>
    #     where grad[i] is dL/dwi
    # '''  
    # # Your implementation here
    # onesCol = np.ones([x.shape[0], 1])
    # X_aug = np.concatenate((onesCol, x), axis=1) 
    # w_aug = np.insert(W.flatten(), 0, b)

    # dL = gradMSEHelper(X_aug, w_aug, y, reg)
    # return dL

def gradMSEHelper(X_aug, w_aug, y, reg):
    # L = L_loss + L_reg
    # dL_loss = 2/N (transpose(X_aug) * (X_aug * w_aug - y)
    N = X_aug.shape[0]
    dL_loss = 2/N * np.matmul(np.transpose(X_aug), np.matmul(X_aug, w_aug) - y[:, 0])
    dL_reg = reg * w_aug
    dL_reg[0] = 0
    dL = dL_loss + dL_reg
    return dL

None


In [ ]:
def grad_descent(W, b, x, y, alpha, epochs, reg, error_tol=10**(-7), lossType="MSE"):
    if (lossType=='CE'):
        for _ in range(epochs):
            dW, db = gradCE(W, b, x, y, reg)
            W = W - alpha*dW
            b = b - alpha*db
            if (np.linalg.norm(alpha*dW)<=error_tol):
                break
            loss.append(crossEntropyLoss(W, b, x, y, reg))
    elif (lossType == "MSE"):
        for i in range(epochs):
            dW, db = gradMSE(W, b, x, y, reg)
            W = W - alpha * dW
            b = b - alpha * db
            if (np.linalg.norm(alpha * dW) <= error_tol):
                break
            loss.append(MSE(W, b, x, y, reg))
      return W, b

In [ ]:
W = 2*np.random.random_sample((784, 1))-1
b = 0
trainLoss = []
validLoss = []
testLoss = []
trainAccuracy = []
validAccuracy = []
testAccuracy = []
xTrain = trainData
xValid = validData
xTest = testData
yTrain = trainTarget
yValid = validTarget
yTest = testTarget
reg = 0.01

def grad_descent(W, b, x, y, alpha, epochs, reg, error_tol=10**(-7), lossType="MSE"):
    if (lossType=='CE'):
        for _ in range(epochs):
            dW, db = gradCE(W, b, x, y, reg)
            W = W - alpha*dW
            b = b - alpha*db
            if (np.linalg.norm(alpha*dW)<=error_tol):
                break
            loss.append(crossEntropyLoss(W, b, x, y, reg))
    elif (lossType == "MSE"):
        for i in range(epochs):
            dW, db = gradMSE(W, b, xTrain, yTrain, reg)
            W = W - alpha * dW
            b = b - alpha * db
            if (np.linalg.norm(alpha * dW) <= error_tol):
                break
            trainLoss.append(MSE(W, b, x, y, reg))
            validLoss.append(MSE(W, b, xValid, yValid, reg))
            testLoss.append(MSE(W, b, xTest, yTest, reg))
            yHatTrain = np.matmul(x, W) + b
            yHatValid = np.matmul(xValid, W) + b
            yHatTest = np.matmul(xTest, W) + b
            for i in range(len(yHatTrain)):
              if yHatTrain[i] >= 0.5:
                yHatTrain[i] = 1
              else:
                yHatTrain[i] = 0
            for i in range(len(yHatValid)):
              if yHatValid[i] >= 0.5:
                yHatValid[i] = 1
              else:
                yHatValid[i] = 0
            for i in range(len(yHatTest)):
              if yHatTest[i] >= 0.5:
                yHatTest[i] = 1
              else:
                yHatTest[i] = 0
            trainSuccesses = 0
            validSuccesses = 0
            testSuccesses = 0
            for i in range(len(yHatTrain)):
              if yHatTrain[i] == yTrain[i]:
                trainSuccesses += 1
            for i in range(len(yHatValid)):
              if yHatValid[i] == yValid[i]:
                validSuccesses += 1
            for i in range(len(yHatTest)):
              if yHatTest[i] == yTest[i]:
                testSuccesses += 1
            trainAccuracy.append(float(trainSuccesses)/len(yHatTrain))
            validAccuracy.append(float(validSuccesses)/len(yHatValid))
            testAccuracy.append(float(testSuccesses)/len(yHatTest))
    return W, b

wFit, bFit = grad_descent(W, b, xTrain, yTrain, alpha=0.005, epochs = 5000, reg = 0.1)

print(MSE(wFit, bFit, xTrain, yTrain, 0))

t = np.arange(len(trainLoss))

fig, axes = plt.subplots(1, 2)
plt.suptitle('Alpha = 0.005, Lambda = 0.1')
axes[0].plot(t, trainLoss)
axes[0].plot(t, validLoss)
axes[0].plot(t, testLoss)
axes[1].plot(t, trainAccuracy)
axes[1].plot(t, validAccuracy)
axes[1].plot(t, testAccuracy)

axes[0].legend(['Training', 'Validation', 'Test'], loc='upper right')
axes[1].legend(['Training', 'Validation', 'Test'], loc='lower right')

0.39006643163948773


In [ ]:
xTrain = trainData
xValid = validData
xTest = testData
yTrain = trainTarget
yValid = validTarget
yTest = testTarget

onesCol = np.ones([xTrain.shape[0], 1])
xTilde = np.concatenate((onesCol, xTrain), axis=1)
wStar = np.matmul(np.linalg.inv(np.matmul(np.transpose(xTilde), xTilde)), np.matmul(np.transpose(xTilde), y))
yHatTrain = np.matmul(xTrain, wStar[1:]) + wStar[0]
yHatValid = np.matmul(xValid, wStar[1:]) + wStar[0]
yHatTest = np.matmul(xTest, wStar[1:]) + wStar[0]
for i in range(len(yHatTrain)):
  if yHatTrain[i] >= 0.5:
    yHatTrain[i] = 1
  else:
    yHatTrain[i] = 0
for i in range(len(yHatValid)):
  if yHatValid[i] >= 0.5:
    yHatValid[i] = 1
  else:
    yHatValid[i] = 0
for i in range(len(yHatTest)):
  if yHatTest[i] >= 0.5:
    yHatTest[i] = 1
  else:
    yHatTest[i] = 0
trainSuccesses = 0
validSuccesses = 0
testSuccesses = 0
for i in range(len(yHatTrain)):
  if yHatTrain[i] == yTrain[i]:
    trainSuccesses += 1
for i in range(len(yHatValid)):
  if yHatValid[i] == yValid[i]:
    validSuccesses += 1
for i in range(len(yHatTest)):
  if yHatTest[i] == yTest[i]:
    testSuccesses += 1
print(float(trainSuccesses)/len(yHatTrain))
print(float(validSuccesses)/len(yHatValid))
print(float(testSuccesses)/len(yHatTest))

print(MSE(wStar[1:], wStar[0], xTrain, yTrain, 0))

0.9937142857142857
0.96
0.9448275862068966
0.01870405565313653


In [ ]:
def crossEntropyLoss(W, b, x, y, reg):
    # Your implementation here

In [ ]:
def gradCE(W, b, x, y, reg):
    # Your implementation here

In [ ]:
def buildGraph(loss="MSE"):
	#Initialize weight and bias tensors
	tf.set_random_seed(421)

	if loss == "MSE":
	# Your implementation
	
	elif loss == "CE":
	#Your implementation here